In [4]:
# mount drive and run all
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# check connnection
!ls "/content/drive/My Drive/app_UNOSAT/src"

funcs_deep_learning.py	prepare_for_network.py	rasterize.py
__init__.py		preprocessing.py	run_pipeline.py
load_data.py		__pycache__		utils.py


In [13]:
!pip install keras_unet 
!pip install geopandas 
!pip install rasterio
!pip install pyproj
!pip install selenium

     |████████████████████████████████| 911kB 2.6MB/s 


In [8]:
from datetime import datetime
import os
from os.path import join
from pprint import pprint
import sys

import warnings
import yaml

import h5py
#import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras import callbacks as keras_cb 
from keras.optimizers import Adam
from keras.utils import Sequence
import matplotlib.pyplot as plt
import numpy as np

#tf.logging.set_verbosity(tf.logging.ERROR)

# # see: https://github.com/karolzak/keras-unet
from keras_unet.models import satellite_unet 
from keras_unet.losses import jaccard_distance
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.utils import get_augmented

print(sys.executable)
sys.path.append("/content/drive/My Drive/app_UNOSAT")
sys.path.append("/content/drive/My Drive/app_UNOSAT/src")

from src import load_data as ld
from src import preprocessing as pp
from src import prepare_for_network as pfn
from src import utils as utl

Using TensorFlow backend.


/usr/bin/python3


In [0]:
# local paths, set your own path here
path_main = '/content/drive/My Drive/app_UNOSAT'
path_data_main = utl.make_dir(join(path_main, "data"))

#### put data in these dirs

# I choose:
# Training: Mosul, Najaf, Nasiryah
# Validation: Souleimaniye
# Test: Bagdad, Kirkouk, Samawah, Tikrit (no labels present)
path_data_local_train = join(path_data_main, 'Train_Dataset')
path_data_local_val = join(path_data_main, 'Validation_Dataset')
path_data_local_test = join(path_data_main, 'Evaluation_Dataset')

path_model = utl.make_dir(join(path_data_main, 
                               'model_{}'.format(utl.datetime_now())))
path_checkpoints =  utl.make_dir(join(path_model, 'checkpoints'))

In [0]:
# other dirs are made automatically
dir_temp_data =  utl.make_dir(join(path_data_main, 'data_temp'))
dir_temp_train = utl.make_dir(join(dir_temp_data, 'train'))
dir_temp_val = utl.make_dir(join(dir_temp_data, 'val'))
dir_temp_dev = utl.make_dir(join(dir_temp_data, 'data_dev'))
dir_temp_eval = utl.make_dir(join(dir_temp_data, 'evaluation'))
dir_temp_plots = utl.make_dir(join(dir_temp_data, 'plots_preprocessing'))

In [0]:
# parameters
quantile_clip_max = 0.999
dtype = np.uint16
#dtype = np.float32

inv_stride = 2
depth_img = 8
stride_mode = [1,4][1] # = 4

patch_size= (512, 512)

# Load and preprocess

In [0]:
file_hfd5_train_X = join(dir_temp_train, 'MosNajNaS_20191223_X_{}.hdf5'.format(patch_size[0]))
file_hfd5_train_Y = join(dir_temp_train, 'MosNajNaS_20191223_Y_{}.hdf5'.format(patch_size[0]))

if not os.path.exists(file_hfd5_train_X):
  t0 = time.time()
  dict_paths_train = ld.paths_in_dict(path_data_local_train)
  pp.preprocces_data(dict_paths_train, file_hfd5_train_X, quantile_clip_max, patch_size, dtype, dir_temp_plots,
                      apply_log=True, a_l=1, b_l=0.2)
  pp.preprocess_shape(dict_paths_train, file_hfd5_train_Y, patch_size, dtype_labels)

  print('takes {} mins'.format(round((time.time() - t0)/60)))

In [0]:
file_hfd5_val_X = join(dir_temp_val, 'Sou_20191223_X_{}.hdf5'.format(patch_size[0]))
file_hfd5_val_Y = join(dir_temp_val, 'Sou_20191223_Y_{}.hdf5'.format(patch_size[0]))

if not os.path.exists(file_hfd5_val_X):
  t0 = time.time()
  dict_paths_val = ld.paths_in_dict(path_data_local_val)
  pp.preprocces_data(dict_paths_val, file_hfd5_val_X, quantile_clip_max, patch_size, dtype, dir_temp_plots,
                    apply_log=True, a_l=a_l, b_l=b_l)
  pp.preprocess_shape(dict_paths_val, file_hfd5_val_Y, patch_size, dtype_labels)

  print('takes {} mins'.format(round((time.time() - t0)/60)))

# Prepare for network

In [40]:
file_hfd5_train_X_patches = join(dir_temp_train,  'MosNajNaS_20191224_X_{}_patches.hdf5'.format(patch_size[0]))
file_hfd5_train_Y_patches = join(dir_temp_train, 'MosNajNaS_20191224_Y_{}_patches.hdf5'.format(patch_size[0]))
file_hfd5_val_X_patches = join(dir_temp_val,  'Sou_20191224_X_{}_patches.hdf5'.format(patch_size[0]))
file_hfd5_val_Y_patches = join(dir_temp_val, 'Sou_20191224_Y_{}_patches.hdf5'.format(patch_size[0]))

file_hfd5_train_X_patches2 = join(dir_temp_train,  'MosNajNaS_20191222_X_{}_patches.hdf5'.format(patch_size[0]))
file_hfd5_train_Y_patches2 = join(dir_temp_train, 'MosNajNaS_20191222_Y_{}_patches.hdf5'.format(patch_size[0]))
file_hfd5_val_X_patches2 = join(dir_temp_val,  'Sou_20191222_X_{}_patches.hdf5'.format(patch_size[0]))
file_hfd5_val_Y_patches2 = join(dir_temp_val, 'Sou_20191222_Y_{}_patches.hdf5'.format(patch_size[0]))


name_dset_train_X = 'all_train_data_X'
dataset_kind_train_X = 'data'
name_dset_train_Y = 'all_train_data_Y'
dataset_kind_train_Y = 'labels'

name_dset_val_X = 'all_val_data_X'
dataset_kind_val_X = 'data'
name_dset_val_Y = 'all_val_data_Y'
dataset_kind_val_Y = 'labels'

# train
#if not os.path.exists(file_hfd5_train_X_patches):
pfn.prepare_data_for_nn_pipeline(file_hfd5_train_X,
                                  file_hfd5_train_X_patches,
                                  patch_size,
                                  stride_mode, 
                                  name_dset_train_X,
                                  dtype,
                                  inv_stride,
                                  dataset_kind_train_X,
                                  depth_img,
                                   compression_opts=9)

if not os.path.exists(file_hfd5_train_Y_patches):
  pfn.prepare_data_for_nn_pipeline(file_hfd5_train_Y, 
                                  file_hfd5_train_Y_patches,
                                  patch_size,
                                  stride_mode, 
                                  name_dset_train_Y,
                                  dtype,
                                  inv_stride,
                                  dataset_kind_train_Y,
                                   compression_opts=9)
# val
if not os.path.exists(file_hfd5_val_X_patches):
  pfn.prepare_data_for_nn_pipeline(file_hfd5_val_X,
                                  file_hfd5_val_X_patches,
                                  patch_size,
                                  stride_mode, 
                                  name_dset_val_X,
                                  dtype,
                                  inv_stride,
                                  dataset_kind_val_X,
                                  depth_img,
                                   compression_opts=9)

if not os.path.exists(file_hfd5_val_Y_patches):
  pfn.prepare_data_for_nn_pipeline(file_hfd5_val_Y, 
                                  file_hfd5_val_Y_patches,
                                  patch_size,
                                  stride_mode, 
                                  name_dset_val_Y,
                                  dtype,
                                  inv_stride,
                                  dataset_kind_val_Y,
                                   compression_opts=9)

Mosul
(11264, 11264, 8)
dim_str: _
dset2.shape: (484, 512, 512, 8)
idx i,j: 0 484
dim_str: x
dset2.shape: (990, 512, 512, 8)
idx i,j: 484 990
dim_str: y
dset2.shape: (1496, 512, 512, 8)
idx i,j: 990 1496
dim_str: xy
dset2.shape: (2025, 512, 512, 8)
idx i,j: 1496 2025
Najaf
(11264, 11264, 8)
dim_str: _
dset2.shape: (2509, 512, 512, 8)
idx i,j: 2025 2509
dim_str: x
dset2.shape: (3015, 512, 512, 8)
idx i,j: 2509 3015
dim_str: y
dset2.shape: (3521, 512, 512, 8)
idx i,j: 3015 3521
dim_str: xy
dset2.shape: (4050, 512, 512, 8)
idx i,j: 3521 4050
Nasiryah
(11264, 11264, 8)
dim_str: _
dset2.shape: (4534, 512, 512, 8)
idx i,j: 4050 4534
dim_str: x
dset2.shape: (5040, 512, 512, 8)
idx i,j: 4534 5040
dim_str: y


OSError: ignored

In [0]:
file_hfd5_train_X_patches_large = join(dir_temp_train,  'MosNajNaS_20191224_X_{}_patches.hdf5'.format(patch_size[0]))

f1 =  h5py.File(file_hfd5_train_X_patches, 'r') 
f2 = h5py.File(file_hfd5_train_X_patches_large, 'w')

f2[name_dset_train_X] = f1[name_dset_train_X][:].astype('float32')
f1.close()
f2.close()
    # with h5py.File(file_hfd5_train_X_patches_large, 'w') as f2:
    #   dset2 = f2.create_dataset(name_dset_train_X,
    #                       dataset_X.shape,
    #                       dtype=dtype,
    #                       #maxshape=maxshape,
    #                       chunks=True,
    #                       compression="gzip",
    #                       compression_opts=9
    #                       )
    #   dset2[name_dset_train_X] = dataset_X.astype(np.float32)

In [38]:
with h5py.File(file_hfd5_train_X_patches, 'r') as f:
  dset = f['all_train_data_X']
  print(dset.dtype)

OSError: ignored

# Deep learning

In [0]:
# params
BS = 4
EPOCHS = 200
num_layers = 2
INIT_LR = 1e-3

# K.set_floatx('float16')
# K.set_epsilon(1e-4) #default is 1e-7

In [30]:
IMAGE_DIMS = patch_size + (depth_img,)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = satellite_unet(IMAGE_DIMS, num_layers=num_layers)
print(IMAGE_DIMS)

(512, 512, 8)


In [0]:
class My_Custom_Generator(Sequence):
    def __init__(self, file_hdf5_X, file_hdf5_Y, batch_size, name_dset_X, name_dset_Y,
                 dtype):
        self.file_hdf5_X = file_hdf5_X
        self.file_hdf5_Y = file_hdf5_Y
        self.batch_size = batch_size
        self.name_dset_X = name_dset_X
        self.name_dset_Y = name_dset_Y
        self.dtype = dtype

    def __len__(self):
        with h5py.File(self.file_hdf5_Y, 'r') as f1_Y:
            dataset_Y = f1_Y[self.name_dset_Y]
            len_dataset_Y = dataset_Y.shape[0]
        return (np.ceil(len_dataset_Y) / float(self.batch_size)).astype(np.int)

    def __getitem__(self, idx):
        t0=time.time()
        with h5py.File(self.file_hdf5_X, 'r') as f1_X:
            dataset_X = f1_X[self.name_dset_X]
            batch_x = dataset_X[idx * self.batch_size: (idx + 1) * self.batch_size, :, :,
                      :] / utl.get_nmax_dtype(self.dtype)

        with h5py.File(self.file_hdf5_Y, 'r') as f1_Y:
            dataset_Y = f1_Y[self.name_dset_Y]
            batch_y = dataset_Y[idx * self.batch_size: (idx + 1) * self.batch_size, :, :]
        print('takes', round(time.time() - t0, 3), 'sec')
        return (batch_x.astype(np.float32),
                batch_y.reshape(batch_y.shape[0], batch_y.shape[1], batch_y.shape[2], 1).astype(np.float32))

In [35]:
file_hdf5_train_X_nn = file_hfd5_train_X_patches
file_hdf5_train_Y_nn = file_hfd5_train_Y_patches

file_hdf5_val_X_nn = file_hfd5_val_X_patches
file_hdf5_val_Y_nn = file_hfd5_val_Y_patches


my_training_batch_gen = My_Custom_Generator(file_hdf5_train_X_nn, file_hdf5_train_Y_nn, 
                                            BS, 
                                            'all_train_data_X', 'all_train_data_Y', 
                                            dtype)
my_val_batch_gen = My_Custom_Generator(file_hdf5_val_X_nn, file_hdf5_val_Y_nn, BS, 
                                            'all_val_data_X', 'all_val_data_Y', 
                                       dtype)

print(len(my_training_batch_gen), len(my_val_batch_gen))

1518 506


In [36]:
for i, arr in enumerate(my_training_batch_gen):
    print(arr[0].shape, arr[1].shape)
    print('data:',np.min(arr[0]), np.max(arr[0]), np.median(arr[0]), np.mean(arr[0]))
    print('labels:',np.min(arr[1]), np.max(arr[1]), np.median(arr[1]), np.mean(arr[1]))
    print('type data:', type(arr[0][0][0][0][0]), type(arr[1][0][0][0][0]))
    print()
    if i>0:
      break

OSError: ignored

In [0]:
#checkpoints
filepath_chk = None
for i, file_chk in enumerate(os.listdir(path_checkpoints)): 
    if i == len(os.listdir(path_checkpoints)) - 1:
          filepath_chk = join(path_checkpoints, file_chk)
          print('load checkpoint: {}'.format(filepath_chk))
          print('dirname:', os.path.dirname(filepath_chk))

if filepath_chk is not None and False:
    print('loading model..')
    initial_epoch = dl.get_init_epoch(filepath_chk)
    model = load_model(filepath_chk,
                  custom_objects={"iou": iou,
                                  "iou_thresholded": iou_thresholded})
else:
#   model = model_init
    initial_epoch = 0
    print('compile model...')
    model.compile(loss="binary_crossentropy",
                      optimizer=opt,
                      metrics=[iou, iou_thresholded])
    
fn_femplate = "best-weights-{epoch:02d}-{iou_thresholded:.3f}-{val_iou_thresholded:.3f}.hdf5"
filepath_chk_new = join(path_checkpoints, fn_femplate)    
print('initial_epoch:', initial_epoch)
print('use checkpoint file:', filepath_chk_new)

In [0]:
# callbacks
cb_chk = keras_cb.ModelCheckpoint(filepath_chk_new, monitor='iou_thresholded', 
                                  verbose=1, save_best_only=True, mode='max')
cb_es = keras_cb.EarlyStopping(patience=5)
callbacks_list = [cb_chk, cb_es]

In [0]:
dict_hyperparms = {'BS':BS, 'patch_size':patch_size, 
                   'num_layers':num_layers, 'inv_stride':inv_stride, 
                   "INIT_LR":INIT_LR, 'depth_img':depth_img, 
                   'stride_mode':stride_mode, 'val':'Sou', 
                   'quantile_clip_max':quantile_clip_max,
                   'dtype':dtype}
utl.save_yaml_parms(path_model, dict_hyperparms, 'hyperparms.yaml')

In [0]:
H = model.fit_generator(generator=my_training_batch_gen, 
                        steps_per_epoch=len(my_training_batch_gen),
                        #validation_data=my_val_batch_gen,
                        #validation_steps=len(my_val_batch_gen),
                        epochs=EPOCHS,
                        callbacks=callbacks_list,
                        verbose=1,
                        initial_epoch=initial_epoch
                        )

In [0]:
model_json = model.to_json()
path_json_file = join(path_model, "model_{}.json".format(utl.datetime_now()))
with open(path_json_file, 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(join(path_model, "model_epochs_{}.h5".format(utl.datetime_now())))

In [0]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize=(12,9))
plt.plot(H.history["iou_thresholded"], label="train_iou_threshold")
plt.plot(H.history["val_iou_thresholded"], label="val_iou_threshold")

subtitle = "BS={}-patch_size={}-num_layers={}".format(BS, str(patch_size), num_layers)
plt.title("Train and Val iuo_thresholded  \newlinw \tiny{}".format(subtitle))
plt.subtitle
plt.xlabel("Epoch #")
plt.ylabel("iou_thresholded")
plt.legend(loc="lower left")
file_plot_train_info = join(path_model, 'info_train_{}.png'.format(utl.datetime_now()))
plt.savefig(file_plot_train_info)


## evaluation

### def

In [0]:
filepath_model = ''


In [0]:
model = load_model(filepath_model, custom_objects={"iou": iou, "iou_thresholded":iou_thresholded})

In [0]:
# predictions
area = 'Sou'
shape_x = 
shape_y = 

org_img_size = ...


s_x = size_sub_sample[0]
s_y = size_sub_sample[1]
stride_x = s_x
stride_y = s_

with h5py.File(file_val_predict_X, 'w') as f2_X:
    dset2 = f2_X.create_dataset('predict_{}'.format(area), 
                                (shape_x, shape_y),
                                dtype=dtype,
                                maxshape=maxshape,
                                chunks=True)
    
    with h5py.File(file_val_split_hfd5_X, 'r') as f1_X:
        dataset_X  = f1_X['all_val_data_X']
        #TODO: use metadata to split '_', 'x', 'y', 'xy'
        metadata = f1_X.metadata
        
        for dim_str in ['_', 'x', 'y', 'xy']:
            org_img_size = ...
            dataset_dim =...
        
            i_max = (org_img_size[0] // stride) + 1 - (size // stride)
            j_max = (org_img_size[1] // stride) + 1 - (size // stride)

            k = 0
            for i in range(i_max):
                for j in range(j_max):
                    dset2[i*stride_x : i*stride_x+s_x, j*stride_y : j*stride_y+s_y] = model.predict(dataset_dim[k, :, :, :])
                    k += 1       
 

In [0]:
#TODO: zero unpadding

In [0]:
with h5py.File(file_val_split_hfd5_Y, 'r') as f1_Y:
    dataset_Y  = f1_Y['all_val_data_Y']

In [0]:
)